<a href="https://colab.research.google.com/github/nanfang-wuyu/ML4NLP1_UZH/blob/main/Assignment%202/word_embeddings_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
%matplotlib inline

In [107]:
# Optional for word correction
# %pip install textblob
# %pip install pyspellchecker

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Source: [https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html#exercise-computing-word-embeddings-continuous-bag-of-words](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html#exercise-computing-word-embeddings-continuous-bag-of-words)

# Word Embeddings: Encoding Lexical Semantics

Word embeddings are dense vectors of real numbers, one per word in your
vocabulary. In NLP, it is almost always the case that your features are
words! But how should you represent a word in a computer? You could
store its ascii character representation, but that only tells you what
the word *is*, it doesn't say much about what it *means* (you might be
able to derive its part of speech from its affixes, or properties from
its capitalization, but not much). Even more, in what sense could you
combine these representations? We often want dense outputs from our
neural networks, where the inputs are $|V|$ dimensional, where
$V$ is our vocabulary, but often the outputs are only a few
dimensional (if we are only predicting a handful of labels, for
instance). How do we get from a massive dimensional space to a smaller
dimensional space?

How about instead of ascii representations, we use a one-hot encoding?
That is, we represent the word $w$ by

\begin{align}\overbrace{\left[ 0, 0, \dots, 1, \dots, 0, 0 \right]}^\text{|V| elements}\end{align}

where the 1 is in a location unique to $w$. Any other word will
have a 1 in some other location, and a 0 everywhere else.

There is an enormous drawback to this representation, besides just how
huge it is. It basically treats all words as independent entities with
no relation to each other. What we really want is some notion of
*similarity* between words. Why? Let's see an example.

Suppose we are building a language model. Suppose we have seen the
sentences

* The mathematician ran to the store.
* The physicist ran to the store.
* The mathematician solved the open problem.

in our training data. Now suppose we get a new sentence never before
seen in our training data:

* The physicist solved the open problem.

Our language model might do OK on this sentence, but wouldn't it be much
better if we could use the following two facts:

* We have seen  mathematician and physicist in the same role in a sentence. Somehow they
  have a semantic relation.
* We have seen mathematician in the same role  in this new unseen sentence
  as we are now seeing physicist.

and then infer that physicist is actually a good fit in the new unseen
sentence? This is what we mean by a notion of similarity: we mean
*semantic similarity*, not simply having similar orthographic
representations. It is a technique to combat the sparsity of linguistic
data, by connecting the dots between what we have seen and what we
haven't. This example of course relies on a fundamental linguistic
assumption: that words appearing in similar contexts are related to each
other semantically. This is called the `distributional
hypothesis <https://en.wikipedia.org/wiki/Distributional_semantics>`__.


# Getting Dense Word Embeddings

How can we solve this problem? That is, how could we actually encode
semantic similarity in words? Maybe we think up some semantic
attributes. For example, we see that both mathematicians and physicists
can run, so maybe we give these words a high score for the "is able to
run" semantic attribute. Think of some other attributes, and imagine
what you might score some common words on those attributes.

If each attribute is a dimension, then we might give each word a vector,
like this:

\begin{align}q_\text{mathematician} = \left[ \overbrace{2.3}^\text{can run},
   \overbrace{9.4}^\text{likes coffee}, \overbrace{-5.5}^\text{majored in Physics}, \dots \right]\end{align}

\begin{align}q_\text{physicist} = \left[ \overbrace{2.5}^\text{can run},
   \overbrace{9.1}^\text{likes coffee}, \overbrace{6.4}^\text{majored in Physics}, \dots \right]\end{align}

Then we can get a measure of similarity between these words by doing:

\begin{align}\text{Similarity}(\text{physicist}, \text{mathematician}) = q_\text{physicist} \cdot q_\text{mathematician}\end{align}

Although it is more common to normalize by the lengths:

\begin{align}\text{Similarity}(\text{physicist}, \text{mathematician}) = \frac{q_\text{physicist} \cdot q_\text{mathematician}}
   {\| q_\text{\physicist} \| \| q_\text{mathematician} \|} = \cos (\phi)\end{align}

Where $\phi$ is the angle between the two vectors. That way,
extremely similar words (words whose embeddings point in the same
direction) will have similarity 1. Extremely dissimilar words should
have similarity -1.


You can think of the sparse one-hot vectors from the beginning of this
section as a special case of these new vectors we have defined, where
each word basically has similarity 0, and we gave each word some unique
semantic attribute. These new vectors are *dense*, which is to say their
entries are (typically) non-zero.

But these new vectors are a big pain: you could think of thousands of
different semantic attributes that might be relevant to determining
similarity, and how on earth would you set the values of the different
attributes? Central to the idea of deep learning is that the neural
network learns representations of the features, rather than requiring
the programmer to design them herself. So why not just let the word
embeddings be parameters in our model, and then be updated during
training? This is exactly what we will do. We will have some *latent
semantic attributes* that the network can, in principle, learn. Note
that the word embeddings will probably not be interpretable. That is,
although with our hand-crafted vectors above we can see that
mathematicians and physicists are similar in that they both like coffee,
if we allow a neural network to learn the embeddings and see that both
mathematicians and physicists have a large value in the second
dimension, it is not clear what that means. They are similar in some
latent semantic dimension, but this probably has no interpretation to
us.


In summary, **word embeddings are a representation of the *semantics* of
a word, efficiently encoding semantic information that might be relevant
to the task at hand**. You can embed other things too: part of speech
tags, parse trees, anything! The idea of feature embeddings is central
to the field.


# Word Embeddings in Pytorch

Before we get to a worked example and an exercise, a few quick notes
about how to use embeddings in Pytorch and in deep learning programming
in general. Similar to how we defined a unique index for each word when
making one-hot vectors, we also need to define an index for each word
when using embeddings. These will be keys into a lookup table. That is,
embeddings are stored as a $|V| \times D$ matrix, where $D$
is the dimensionality of the embeddings, such that the word assigned
index $i$ has its embedding stored in the $i$'th row of the
matrix. In all of my code, the mapping from words to indices is a
dictionary named word\_to\_ix.

The module that allows you to use embeddings is torch.nn.Embedding,
which takes two arguments: the vocabulary size, and the dimensionality
of the embeddings.

To index into this table, you must use torch.LongTensor (since the
indices are integers, not floats).




In [46]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [47]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings # randomly initial
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward0>)


In [48]:
embeds(torch.LongTensor([1]))

tensor([[-0.1661, -1.5228,  0.3817, -1.0276, -0.5631]],
       grad_fn=<EmbeddingBackward0>)

# An Example: N-Gram Language Modeling

Recall that in an n-gram language model, given a sequence of words
$w$, we want to compute

\begin{align}P(w_i | w_{i-1}, w_{i-2}, \dots, w_{i-n+1} )\end{align}

Where $w_i$ is the ith word of the sequence.

In this example, we will compute the loss function on some training
examples and update the parameters with backpropagation.




In [49]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}


class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # n probs, 1 target idx
        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


[518.7701461315155, 516.090772151947, 513.4266452789307, 510.77938985824585, 508.14692997932434, 505.5277931690216, 502.9213914871216, 500.32760977745056, 497.74508023262024, 495.1726322174072]


# Exercise: Computing Word Embeddings: Continuous Bag-of-Words

The Continuous Bag-of-Words model (CBOW) is frequently used in NLP deep
learning. It is a model that tries to predict words given the context of
a few words before and a few words after the target word. This is
distinct from language modeling, since CBOW is not sequential and does
not have to be probabilistic. Typcially, CBOW is used to quickly train
word embeddings, and these embeddings are used to initialize the
embeddings of some more complicated model. Usually, this is referred to
as *pretraining embeddings*. It almost always helps performance a couple
of percent.

The CBOW model is as follows. Given a target word $w_i$ and an
$N$ context window on each side, $w_{i-1}, \dots, w_{i-N}$
and $w_{i+1}, \dots, w_{i+N}$, referring to all context words
collectively as $C$, CBOW tries to minimize

\begin{align}-\log p(w_i | C) = -\log \text{Softmax}(A(\sum_{w \in C} q_w) + b)\end{align}

where $q_w$ is the embedding for word $w$.

Implement this model in Pytorch by filling in the class below. Some
tips:

* Think about which parameters you need to define.
* Make sure you know what shape each operation expects. Use .view() if you need to
  reshape.




In [50]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
print(data[:5])




def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

# data[i]: i-th contexts-target pair
# data[i][0]: 4 contexts (list)
# data[i][1]: target (text)
make_context_vector(data[0][0], word_to_ix)  # example

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]


tensor([21, 10, 26, 28])

## Model

In [51]:
class CBOW(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(2 * context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)


    def forward(self, inputs):
        embeds = self.embeddings(inputs).view(-1, self.linear1.in_features)
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

# create your model and train.  here are some functions to help you make
# the data ready for use by your module


## Data Preprocessing

In [52]:
!gdown 1foE1JuZJeu5E_4qVge9kExzhvF32teuF
!gdown 13IWXrTjGTrfCd9l7dScZVO8ZvMicPU75

zsh:1: command not found: gdown
zsh:1: command not found: gdown


### Load Data

In [53]:
with open(f'scifi_reduced.txt') as f:
    text_scifi = f.read()
import pandas as pd
df_hotel = pd.read_csv('tripadvisor_hotel_reviews_reduced.csv')

In [54]:
text_scifi[:20]

' A chat with the edi'

In [55]:
df_hotel[:5]

,Review,Rating
0,fantastic service large hotel caters business ...,5
1,"great hotel modern hotel good location, locate...",4
2,3 star plus glasgowjust got 30th november 4 da...,4
3,nice stayed hotel nov 19-23. great little bout...,4
4,great place wonderful hotel ideally located me...,5


In [56]:
text_hotel = df_hotel.drop(columns=["Rating"])

In [57]:
text_hotel[:5]

,Review
0,fantastic service large hotel caters business ...
1,"great hotel modern hotel good location, locate..."
2,3 star plus glasgowjust got 30th november 4 da...
3,nice stayed hotel nov 19-23. great little bout...
4,great place wonderful hotel ideally located me...


1. Data Cleaning, 2. **Character Casing**, 3. **Splitting Sentences** (optional), 4. **Tokenizing Words**, 5. **Dealing with Punctuation**, 6. Expanding Contractions (optional), 7. **Removing Stopwords**, 8. Lemmatization and Stemming (optional), 9. Handling Special Cases, 10. Handling Rare Words and Out-of-Vocabulary Words.

1. Special Characters Cleaning
2. Character Casing
3. Do not split sentences or tokenize words
4. Do not remove stopword, may be beneficial
5. Do not handle Out-of-Vocabulary Words maybe German

### Special Characters Cleaning

In [58]:
# import library: Regular Expression
import re

# Clean the data by removing special characters (punctuation)
def sp_chara_cleaning(text):
    clean_text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
    return clean_text

In [59]:
sp_chara_cleaning("#trap music <123> @gmail! How are you?")

' trap music  123   gmail  How are you '

In [60]:
' trap music  123   gmail  How are you '.split()

['trap', 'music', '123', 'gmail', 'How', 'are', 'you']

### Character Casing

In [61]:
def character_casing(text):
    lower_text = text.lower()
    return lower_text

In [62]:
character_casing("STARBUCKS Starbucks")

'starbucks starbucks'

### Apply Preprocessing Functions

In [63]:
text_hotel[:5]

,Review
0,fantastic service large hotel caters business ...
1,"great hotel modern hotel good location, locate..."
2,3 star plus glasgowjust got 30th november 4 da...
3,nice stayed hotel nov 19-23. great little bout...
4,great place wonderful hotel ideally located me...


In [64]:
text_hotel["Review"] = text_hotel["Review"].apply(sp_chara_cleaning)
text_hotel["Review"] = text_hotel["Review"].apply(character_casing)

In [65]:
# show text after preprocessing
text_hotel[:5]

,Review
0,fantastic service large hotel caters business ...
1,great hotel modern hotel good location locate...
2,3 star plus glasgowjust got 30th november 4 da...
3,nice stayed hotel nov 19 23 great little bout...
4,great place wonderful hotel ideally located me...


In [66]:
text_scifi[:20]

' A chat with the edi'

In [67]:
text_scifi = sp_chara_cleaning(text_scifi)
text_scifi = character_casing(text_scifi)
# show text after preprocessing
text_scifi[:20]

' a chat with the edi'

### Create Dataset For Training

In [68]:
def create_triagrams(i_text, context_size=2):
    i_context_tensor = []
    i_target_tensor = []
    for i in range(context_size, len(i_text) - context_size):
        i_contexts = [i_text[j] for j in range(i - context_size, i + context_size + 1) if i != j]
        i_target = i_text[i]
        i_context_tensor.append(i_contexts)
        i_target_tensor.append(i_target)
    i_target_tensor = torch.LongTensor(i_target_tensor)
    i_context_tensor = torch.LongTensor(i_context_tensor)
    return i_context_tensor, i_target_tensor

#### SCIFI DATASET

In [69]:
words = text_scifi.split()
vocab_scifi = list(set(words))
word_to_ix_scifi = {word: i for i, word in enumerate(vocab_scifi)}
i_text = [word_to_ix_scifi[w] for w in words]
i_context_scifi, i_target_scifi = create_triagrams(i_text, 2)



In [70]:
len(words), len(vocab_scifi), i_target_scifi.shape, i_context_scifi.shape

(7830344, 111577, torch.Size([7830340]), torch.Size([7830340, 4]))

In [71]:

# def create_triagrams(i_text, context_size, i_data):

#     for i in range(context_size, len(i_text) - context_size):
#         idxs = [i_text[j] for j in range(i - context_size, i + context_size + 1)]
#         idxs.pop(len(idxs) // 2)
#         i_target = i_text[i]
#         i_data.append((torch.tensor(idxs, dtype=torch.long), torch.tensor(i_target, dtype=torch.long)))

#     return i_data

#### HOTEL DATASET

In [72]:
words = ' '.join(text_hotel['Review']).split()
vocab_hotel = list(set(words))
word_to_ix_hotel = {word: i for i, word in enumerate(vocab_hotel)}
i_text = [word_to_ix_hotel[w] for w in words]

In [73]:
i_context_hotel_2, i_target_hotel_2 = create_triagrams(i_text, 2)
i_context_hotel_5, i_target_hotel_5 = create_triagrams(i_text, 5)

In [74]:
len(words), len(vocab_hotel)

(1092224, 36160)

In [75]:
i_context_hotel_2.shape, i_target_hotel_2.shape, i_context_hotel_5.shape, i_target_hotel_5.shape

(torch.Size([1092220, 4]),
 torch.Size([1092220]),
 torch.Size([1092214, 10]),
 torch.Size([1092214]))

## Train Model

In [76]:
# device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = torch.device("mps")
device

device(type='mps')

In [77]:
torch.cuda.is_available()

False

In [78]:
import time
import logging

In [79]:
from torch.utils.data import IterableDataset, DataLoader

# Example from 05_intro_to_PyTorch.ipynb
class MyDataset(IterableDataset):
    def __init__(self, data_X, data_y):
        assert len(data_X) == len(data_y)
        self.data_X = data_X.to(device)
        self.data_y = data_y.to(device)

    def __len__(self):
        return len(self.data_X)

    def __iter__(self):
        for i in range(len(self.data_X)):
            yield (self.data_X[i], self.data_y[i])



In [80]:
def createDataLoader(X, y):
  torch.manual_seed(1)
  ridxs = torch.randperm(len(X))
  # ridxs = ridxs[: len(ridxs) // 10]
  sp = len(ridxs)*4//5
  train_set = MyDataset(X[ridxs[:sp]], y[ridxs[:sp]])
  test_set = MyDataset(X[ridxs[sp:]], y[ridxs[sp:]])

  train_loader = DataLoader(train_set, batch_size=64)
  test_loader = DataLoader(test_set, batch_size=64)
  return train_loader, test_loader

### CBOW 2 with Hotel Dataset

In [81]:
logging.basicConfig(filename='training_hotel2.log', level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
train_loader, test_loader = createDataLoader(i_context_hotel_2, i_target_hotel_2)
CONTEXT_SIZE = 2
EMBEDDING_DIM = 50
losses = []
loss_function = nn.NLLLoss()
CBOW2_hotel_model = CBOW(len(vocab_hotel), EMBEDDING_DIM, CONTEXT_SIZE).to(device)
optimizer = optim.SGD(CBOW2_hotel_model.parameters(), lr=0.001)
epochs = 12

In [82]:
for epoch in range(epochs):
    total_loss = 0
    start_time = time.time()
    for batch_num, (i_context, i_target) in enumerate(train_loader):
        # Torch accumulates gradients. Before passing in a
        # new instance, zero out the gradients from the old instance
        CBOW2_hotel_model.zero_grad()

        # Run the forward pass, getting log probabilities over next words
        log_probs = CBOW2_hotel_model(i_context)

        # Compute loss function.
        loss = loss_function(log_probs, i_target)

        # Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Get the Python number from a 1-element Tensor by calling tensor.item()
    info = f"Epoch: {epoch + 1} / {epochs} Loss: {total_loss / len(train_loader):.4f} \
Time: {time.time() - start_time:.2f}s"
    print(info)
    logging.info(info)
torch.save(CBOW2_hotel_model.state_dict(), f'CBOW2_hotel_model_{epochs}.pth')

Epoch: 1 / 12 Loss: 0.4613 Time: 451.50s
Epoch: 2 / 12 Loss: 0.0028 Time: 451.80s
Epoch: 3 / 12 Loss: 0.0015 Time: 451.47s
Epoch: 4 / 12 Loss: 0.0010 Time: 453.90s
Epoch: 5 / 12 Loss: 0.0008 Time: 451.27s
Epoch: 6 / 12 Loss: 0.0006 Time: 450.29s
Epoch: 7 / 12 Loss: 0.0005 Time: 450.82s
Epoch: 8 / 12 Loss: 0.0004 Time: 452.95s
Epoch: 9 / 12 Loss: 0.0004 Time: 454.65s
Epoch: 10 / 12 Loss: 0.0003 Time: 473.36s
Epoch: 11 / 12 Loss: 0.0003 Time: 488.79s
Epoch: 12 / 12 Loss: 0.0003 Time: 477.94s


In [84]:
corr = 0

with torch.no_grad():  # Disable gradient calculation
    for i_context, i_target in test_loader:
        log_probs = CBOW2_hotel_model(i_context)
        _, predicted = torch.max(log_probs, 1)

        # print(list(vocab_hotel)[predicted[0]], list(vocab_hotel)[i_target[0]], [list(vocab_hotel)[id] for id in i_context[0]])
        corr += (predicted == i_target).sum().item()

total = len(test_loader) * test_loader.batch_size

accuracy = round(corr / total, 4)
accuracy

0.0

In [85]:
corr = 0

with torch.no_grad():  # Disable gradient calculation
    for i_context, i_target in train_loader:
        log_probs = CBOW2_hotel_model(i_context)
        _, predicted = torch.max(log_probs, 1)

        # print(list(vocab_hotel)[predicted[0]], list(vocab_hotel)[i_target[0]], [list(vocab_hotel)[id] for id in i_context[0]])
        corr += (predicted == i_target).sum().item()

total = len(train_loader) * train_loader.batch_size

accuracy = round(corr / total, 4)
accuracy

1.0

### CBOW 5 with Hotel Dataset

In [63]:
logging.basicConfig(filename='training_hotel5.log', level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
train_loader, test_loader = createDataLoader(i_context_hotel_5, i_target_hotel_5)
CONTEXT_SIZE = 5
EMBEDDING_DIM = 50
losses = []
loss_function = nn.NLLLoss()
CBOW5_hotel_model = CBOW(len(vocab_hotel), EMBEDDING_DIM, CONTEXT_SIZE).to(device)
optimizer = optim.SGD(CBOW5_hotel_model.parameters(), lr=0.001)
epochs = 12

In [64]:
for epoch in range(epochs):
    total_loss = 0
    start_time = time.time()
    for batch_num, (i_context, i_target) in enumerate(train_loader):

        # Torch accumulates gradients. Before passing in a
        # new instance, zero out the gradients from the old instance
        CBOW5_hotel_model.zero_grad()

        # Run the forward pass, getting log probabilities over next words
        log_probs = CBOW5_hotel_model(i_context)

        # Compute loss function.
        loss = loss_function(log_probs, i_target)

        # Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    info = f"Epoch: {epoch + 1} / {epochs} Loss: {total_loss / len(train_loader):.4f} \
Time: {time.time() - start_time:.2f}s"
    print(info)
    logging.info(info)
#     losses.append(total_loss)
# print(losses)  # The loss decreased every iteration over the training data!
torch.save(CBOW5_hotel_model.state_dict(), 'CBOW5_hotel_model.pth')

Epoch: 1 / 12 Loss: 10.7536 Time: 34.60s
Epoch: 2 / 12 Loss: 10.3630 Time: 34.81s
Epoch: 3 / 12 Loss: 9.8835 Time: 34.83s
Epoch: 4 / 12 Loss: 9.4704 Time: 34.92s
Epoch: 5 / 12 Loss: 9.1407 Time: 34.93s
Epoch: 6 / 12 Loss: 8.8752 Time: 35.89s
Epoch: 7 / 12 Loss: 8.6692 Time: 34.80s
Epoch: 8 / 12 Loss: 8.5112 Time: 34.79s
Epoch: 9 / 12 Loss: 8.3899 Time: 34.88s
Epoch: 10 / 12 Loss: 8.2934 Time: 34.94s
Epoch: 11 / 12 Loss: 8.2134 Time: 35.19s
Epoch: 12 / 12 Loss: 8.1455 Time: 35.90s
[141291.49048137665, 136159.42765903473, 129859.75561237335, 124432.1491651535, 120099.86243057251, 116610.79934835434, 113904.08239984512, 111829.03648710251, 110235.42389297485, 108967.0970029831, 107915.43462085724, 107023.9991517067]


In [65]:
corr = 0

with torch.no_grad():  # Disable gradient calculation
    for i_context, i_target in test_loader:
        log_probs = CBOW5_hotel_model(i_context)
        _, predicted = torch.max(log_probs, 1)

        # print(list(vocab_hotel)[predicted[0]], list(vocab_hotel)[i_target[0]], [list(vocab_hotel)[id] for id in i_context[0]])
        corr += (predicted == i_target).sum().item()

total = len(test_loader) * test_loader.batch_size

accuracy = round(corr / total, 4)
accuracy

0.0371

### CBOW 2 with Scifi Dataset

In [61]:
logging.basicConfig(filename='training_scifi.log', level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
train_loader, test_loader = createDataLoader(i_context_scifi, i_target_scifi)
CONTEXT_SIZE = 2
EMBEDDING_DIM = 50
losses = []
loss_function = nn.NLLLoss()
CBOW2_scifi_model = CBOW(len(vocab_scifi), EMBEDDING_DIM, CONTEXT_SIZE).to(device)
optimizer = optim.SGD(CBOW2_scifi_model.parameters(), lr=0.001)
epochs = 3

In [62]:
for epoch in range(epochs):
    total_loss = 0
    start_time = time.time()
    for batch_num, (i_context, i_target) in enumerate(train_loader):

        # Torch accumulates gradients. Before passing in a
        # new instance, zero out the gradients from the old instance
        CBOW2_scifi_model.zero_grad()

        # Run the forward pass, getting log probabilities over next words
        log_probs = CBOW2_scifi_model(i_context)

        # Compute loss function.
        loss = loss_function(log_probs, i_target)

        # Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Get the Python number from a 1-element Tensor by calling tensor.item()
    info = f"Epoch: {epoch + 1} / {epochs} Loss: {total_loss / len(train_loader):.4f} \
Time: {time.time() - start_time:.2f}s"
    print(info)
    logging.info(info)
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!
torch.save(CBOW2_scifi_model.state_dict(), 'CBOW2_scifi_model.pth')

Epoch: 1 / 3 Loss: 9.0410 Time: 501.97s
Epoch: 2 / 3 Loss: 7.6213 Time: 503.82s
Epoch: 3 / 3 Loss: 7.2408 Time: 501.94s
[860257.098921299, 725172.1176390648, 688966.0479745865]


In [67]:
corr = 0
with torch.no_grad():  # Disable gradient calculation
    for i_context, i_target in test_loader:
        log_probs = CBOW2_scifi_model(i_context)
        _, predicted = torch.max(log_probs, 1)

        # print(list(vocab_hotel)[predicted[0]], list(vocab_hotel)[i_target[0]], [list(vocab_hotel)[id] for id in i_context[0]])
        corr += (predicted == i_target).sum().item()

total = len(test_loader) * test_loader.batch_size

accuracy = round(corr / total, 4)
accuracy

0.1074

# Part 2 Embeddings Evaluation

In [165]:
# Run here only if you load the model from directory.
# If you have a model trained in this jupyter notebook kernel, move ahead. 

# CBOW2_hotel_model = CBOW(len(vocab_hotel), 50, 2).to(device)
# CBOW2_hotel_model.load_state_dict(torch.load('CBOW2_hotel_model.pth', map_location=torch.device('cpu')))
# CBOW2_hotel_model.eval()

CBOW(
  (embeddings): Embedding(51839, 50)
  (linear1): Linear(in_features=200, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=51839, bias=True)
)

In [87]:
import torch.nn as nn

def get_closest_word(word, net, word_to_index, vocabulary, topn=5, device="cpu"):
    net.eval()
    word_distance = []
    emb = net.embeddings
    pdist = nn.PairwiseDistance()
    i = word_to_index[word]
    lookup_tensor_i = torch.tensor([i], dtype=torch.long).to(device)
    v_i = emb(lookup_tensor_i)
    for j in range(len(vocabulary)):
        if j != i:
            lookup_tensor_j = torch.tensor([j], dtype=torch.long).to(device)
            v_j = emb(lookup_tensor_j)
            word_distance.append((vocabulary[j], float(pdist(v_i, v_j))))
            word_distance.sort(key=lambda x: x[1])
    return word_distance[:topn]

In [132]:
words = ' '.join(text_hotel['Review']).split()
vocab_hotel = list(set(words))
word_count =   {word: words.count(word) for word in vocab_hotel[:500]}
word_rank = sorted(word_count.keys(), reverse=True, key= lambda x: word_count[x])

In [133]:
[(w, word_count[w]) for w in word_rank[:10]], 

([('hotels', 2049),
  ('guests', 1056),
  ('extra', 894),
  ('real', 564),
  ('quick', 459),
  ('superb', 393),
  ('expectations', 308),
  ('swimming', 277),
  ('takes', 272),
  ('etc', 233)],)

In [143]:
[(w, word_count[w]) for w in word_rank[80:100]], 

([('swing', 16),
  ('rod', 15),
  ('beutiful', 14),
  ('outrageously', 14),
  ('artus', 13),
  ('subsequently', 13),
  ('mimosas', 13),
  ('cheery', 12),
  ('ceremony', 12),
  ('invaluable', 12),
  ('goers', 12),
  ('frogs', 12),
  ('volley', 11),
  ('trump', 11),
  ('weigh', 11),
  ('forms', 11),
  ('talent', 10),
  ('essentials', 10),
  ('murano', 10),
  ('cadran', 9)],)

In [161]:
chosen_words = ['hotel', 'guests', 'statements', 'takes', 'overcome', 'weigh', 'real', 'quick', 'cheery']
s = f"Chosen words:   \n\
    nouns:  'hotel': {words.count('hotel')}, \n\
            'guests': {words.count('guests')}, \n\
            'statements': {words.count('statements')},\n\
    verbs:  'takes': {words.count('takes')}, \n\
            'overcome': {words.count('overcome')},\n\
            'weigh': {words.count('weigh')},\n\
    adjs:   'real': {words.count('real')}, \n\
            'quick': {words.count('quick')},\n\
            'cheery': {words.count('cheery')},"

In [162]:
print(s)

Chosen words:   
    nouns:  'hotel': 24529, 
            'guests': 1056, 
            'statements': 4,
    verbs:  'takes': 272, 
            'overcome': 7,
            'weigh': 11,
    adjs:   'real': 564, 
            'quick': 459,
            'cheery': 12,


In [166]:
chosen_words_neighbors = [get_closest_word(w, CBOW2_hotel_model, word_to_ix_hotel, vocab_hotel, device="mps") for w in chosen_words]

In [167]:
for i, w in enumerate(chosen_words):
    print(f"{w}:{chosen_words_neighbors[i]}")

hotel:[('complicate', 5.894118785858154), ('ivor', 6.060270309448242), ('1995', 6.177227973937988), ('filtration', 6.2042765617370605), ('path', 6.2997236251831055)]
guests:[('wheelchair', 7.170873165130615), ('therapeutic', 7.434506416320801), ('horizons', 7.5423126220703125), ('sidewalks', 7.564052104949951), ('innkeepers', 7.581796646118164)]
statements:[('immovable', 6.761836051940918), ('odeon', 6.7916388511657715), ('loadsof', 6.919468879699707), ('picky', 7.00703239440918), ('variables', 7.0185675621032715)]
takes:[('wording', 5.869082927703857), ('robably', 6.316221237182617), ('barecelo', 6.470187664031982), ('plantations', 6.486098289489746), ('defintiely', 6.499187469482422)]
overcome:[('sensitively', 6.903243064880371), ('grassy', 7.18086576461792), ('suare', 7.33463716506958), ('disserving', 7.339474201202393), ('robes', 7.344493865966797)]
weigh:[('thur', 6.378139019012451), ('lirr', 6.460971832275391), ('cells', 6.46674919128418), ('pounced', 6.497954845428467), ('recap'